In [11]:
def readDataFromTxt(filePath):
    text = []
    fh = open(filePath,encoding= 'utf8').readlines()
    data = ""
    count = 1
    for line in fh:
        line.replace("\n", '')
        data += line
        if count%4 ==0 :
            text.append(data)
            data = ""
        count += 1
    return text

In [12]:
def deleteAlphabet(word):
    newWord = ""
    index = np.random.randint(0,len(word))
    if index == 0:
        newWord = word[1:]
    elif index == len(word) -1 :
        newWord = word[:len(word)-1]
    else:
        newWord = word[:index] + word[index+1:]
    return newWord  

In [13]:
def changeOrder(word):
    index = np.random.randint(0,len(word))
    secIndex = np.random.randint(0,len(word))
    while (index == secIndex):
        secIndex = np.random.randint(0,len(word))
    strlist = list(word)

    first = ''
    sec = ''
    if strlist[index].isupper():
        sec = strlist[secIndex].upper()
    if strlist[secIndex].isupper():
        first = strlist[index].upper()
    else:
        first = strlist[index].lower()
    strlist[index],strlist[secIndex] = sec,first
    return "".join(strlist)

In [14]:
import string
import random
def insertAlphabet(word):
    index = np.random.randint(0,len(word))
    pre = word[:index]
    post = word[index:]
    randomAlphabet = random.choice(string.ascii_lowercase)
    if word[index].isupper():
        randomAlphabet = randomAlphabet.upper()
        word[index].lower()
    return (pre + randomAlphabet + post)

In [15]:
def substituteAlphabet(word):
    randomAlphabet = random.choice(string.ascii_lowercase)
    index = np.random.randint(0,len(word))
    pre = word[:index]
    post = word[index+1:]
    if word[index][0].isupper():
        randomAlphabet = randomAlphabet.upper()
    if index == 0:
        pre = ''
    elif index == len(word) -1 :
        post = ''
    return (pre + randomAlphabet + post)

In [16]:
from PyDictionary import PyDictionary
import re
import numpy as np
dictionary = PyDictionary()

def syncWordGeneration(word):
    generatedWord = word
    while(dictionary.meaning(generatedWord,True) is not None): 
        operationNumb = np.random.randint(1,5)
        if operationNumb == 1:
            generatedWord  = deleteAlphabet(word)
        elif operationNumb == 2:
            generatedWord = changeOrder(word)
        elif operationNumb == 3:
            generatedWord = insertAlphabet(word)
        else:
            generatedWord = substituteAlphabet(word)
    return generatedWord

In [17]:
import numpy as np

def syncDataListGeneration(dataArray):
    generatedDataList = list()
    for data in dataArray:
        dataList = re.findall("[a-zA-Z]*’?[a-zA-Z]+|[\W]",data)
        orgWords = list(filter(lambda x : x != " ",dataList))
        words = re.findall("[a-zA-Z]*’?[a-zA-Z]+",data)
        randoms = np.random.randint(0,len(words), int(len(words)/20) +1)
        if len(randoms) != 0:
            for index in randoms:
                while(len(words[index]) == 1):
                    index = np.random.randint(0,len(words))
                generatedWord = syncWordGeneration(words[index])
                if words[index] not in orgWords:
                    print(orgWords)
                else :
                    orgIndex = orgWords.index(words[index])
                    orgWords[orgIndex] = generatedWord
        dataString = " ".join(orgWords)

        for i in re.findall('[\W]\s|\s[\W]',dataString):
            dataString = dataString.replace(i,i.strip())
        finalList = dataString.split()

        for id in range(1,int(len(finalList)/18)+1):
            finalList.insert(20*id,'\n')

        print(finalList)
        generatedDataList.append(" ".join(finalList))

    return generatedDataList

In [18]:
from PIL import Image, ImageFont, ImageDraw

fontTypes = ['Calibri','ENGR','Arial','BRUSHSCI','Bahnschrift','Candara','LATINWD','CASTELAR','Corbel','Ebrima','BROADW','Gadugi','BAUHS93',
'Georgia','Verdana','ITCBLKAD','CENTURY','COLONNA','COOPBL','CURLZ___','FELIXTI','FTLTLT','FORTE','FREESCPT','Gabriola','GIGI',
'GLECB','HATTEN','HARLOWSI','HARNGTON','impact','IMPRISHA','INFROMAN','Inkfree','javatext','JOKERMAN',
'ITCKRIST','LHANDW']

fontSizes = [28,18,28,38,30,28,18,25,31,31,26,28,27,31,29,32,32,35,28,32,26,32,34,50,42,28,28,35,33,29,33,31,33,32,30,30,28,30]

def saveImage(data,numb):
    img = Image.new('RGB', (1800, 500), color = (204, 210, 217))
    d = ImageDraw.Draw(img)
    rand = random.randint(0,len(fontTypes)-1)
    fontFamily = "C:\Windows\Fonts\/"+ fontTypes[rand] + ".ttf"
    font = ImageFont.truetype(fontFamily,fontSizes[rand])
    print(count, fontTypes[rand], rand)
    d.text((50,100), data, fill=(0,0,0), font = font, encoding= 'utf8')
    name = "generatedData"+str(numb)+".png"
    img.save(name)

In [19]:
def writeDataToFile(file, data):
        file = open(file,"a")
        for instance in data:
            file.write(instance)
            file.write("\n")
        file.close()    

In [20]:
text = readDataFromTxt("D:\ASSOC\englishSpellingCorrection_imageProcessing\Dataset\Script\sampleScript1.txt")
writeDataToFile('orginalData.txt',text)  
generatedData = syncDataListGeneration(text)
writeDataToFile('generatedData.txt',generatedData)
count = 11
for data in generatedData:
    saveImage(data,count)
    count += 1

['Once', 'I’m', 'finished', 'using', 'the', 'restroom,', 'I', 'take', 'at', 'least', 'a', 'full', 'minute', 'to', 'wash', 'my', 'hands.', 'I', 'stare', 'down', '\n', 'at', 'them', 'the', 'entire', 'time,', 'refusing', 'to', 'look', 'ih', 'the', 'mirror.', 'Avoiding', 'my', 'reflection', 'becomes', 'easier', 'by', 'the', 'day,', '\n', 'but', 'I', 'still', 'catch', 'a', 'glimpse', 'of', 'myself', 'while', 'reaching', 'for', 'a', 'paper', 'towel.', 'No', 'matter', 'how', 'many', 'times', '\n', 'I’ve', 'looked', 'in', 'a', 'mirror,', 'I', 'still', 'haven’t', 'grown', 'used', 'to', 'what', 'I', 'see.', 'I', 'bring', 'my', 'left', 'xhand', '\n', 'up', 'and', 'touch', 'the', 'ocars', 'that', 'run', 'across', 'the', 'left', 'side', 'of', 'my', 'face,', 'over', 'my', 'jaw', 'and', 'down', '\n', 'my', 'neck.', 'They', 'disappear', 'beneath', 'the', 'collar', 'of', 'my', 'shirt,', 'but', 'underneath', 'my', 'clothing,', 'the', 'scars', 'run', 'down', 'the', '\n', 'entire', 'left', 'side', 'of', '

In [ ]:
'''
fontTypes = ['Calibri','ENGR','Arial','BRUSHSCI','Bahnschrift','Candara','LATINWD','CASTELAR','Corbel','Ebrima','BROADW','Gadugi','BAUHS93',
'Georgia','Verdana','ITCBLKAD','CENTURY','COLONNA','COOPBL','CURLZ___','FELIXTI','FTLTLT','FORTE','FREESCPT','Gabriola','GIGI',
'GLECB','HATTEN','HARLOWSI','HARNGTON','impact','IMPRISHA','INFROMAN','Inkfree','javatext','JOKERMAN',
'ITCKRIST','LHANDW']

fontSizes = [28,18,28,38,30,28,18,25,31,31,26,28,27,31,29,32,32,35,28,32,26,32,34,50,42,28,28,35,33,29,33,31,33,32,30,30,28,30]
'''